# Pulling Venue Data from FourSquare API

__Skip to the Good Stuff__

[Importing Libraries](#Importing-Libraries)

[Connecting to FourSquare](#Connecting-to-FourSquare)

[Testing the Code](#Testing-the-Code)

[Making the DataFrame](#Making-the-DataFrame)

[Dropping the Nulls](#Dropping-the-Nulls)

[Sorting by Venue and Distance](#Sorting-by-Venue-and-Distance)

[Obtaining Neighborhood for Each Arcade](#Obtaining-Neighborhood-for-Each-Arcade)

[Pulling the Ratings](#Pulling-the-Ratings)

### Importing Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

! pip install folium==0.5.0
import folium # plotting library

import json

pd.set_option('display.max_rows', None) #To view the full dataframe

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [35]:
atlanta_df = pd.read_csv('atlanta_geodata.csv')
print(len(atlanta_df))
atlanta_df.head(10)

243


,NPU,Neighborhood,hood_longitude,hood_latitude
0,A,Chastain Park,-84.3910383,33.8712146
1,A,Kingswood,-84.405006,33.832115
2,A,Margaret Mitchell,-84.38461443979591,33.781410050000005
3,A,Mt. Paran Parkway,-84.435792,33.867538
4,A,Mt. Paran/Northside,No Data Found,No Data Found
5,A,Paces,-84.48147871405601,33.85700215
6,A,Pleasant Hill,No Data Found,No Data Found
7,A,Randall Mill,-101.8301344,35.0660702
8,A,Tuxedo Park,No Data Found,No Data Found
9,A,West Paces Ferry/Northside,No Data Found,No Data Found


### Connecting to FourSquare

In [36]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [37]:
VERSION = '20180604'
LIMIT = 30

### Testing the Code

In [23]:
#assign variables for the search
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Arcade .... OK!


In [24]:
for index, hood in enumerate(atlanta_df['hood_latitude']):
    if index == 2: #preselected based on venues in initial search
        latitude= atlanta_df['hood_latitude'][index]
        longitude= atlanta_df['hood_longitude'][index]
print(latitude, longitude)

33.781410050000005 -84.38461443979591


In [25]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '61410b07db34ce62720af32a'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': []}}

In [10]:
ident=results['response']['venues'][0]
type(ident)
for dictionary in results['response']['venues']:
    print(dictionary['id'])
    print(dictionary['name'])
    print(dictionary['location']['address'])
    print(dictionary['location']['lat'])
    print(dictionary['location']['lng'])
    print(dictionary['location']['distance'])
    for cat in dictionary['categories']:
        print(cat['id'], cat['name'])

4e752f8e52b1c8e51958e177
Italian Bistro


KeyError: 'address'

### Making the DataFrame

In [38]:
#assign variables for the search
search_query = 'Arcade'
radius = 2000
print(search_query + ' .... OK!')

Arcade .... OK!


In [41]:
atlanta_venues = pd.DataFrame(columns = ['Hood','Venue_ID','Venue_Name','Venue_Distance','Venue_Address','Venue_Latitude','Venue_Longitude','Category_ID','Category_Name'])
atlanta_venues.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name


In [42]:
for index, hood in enumerate(atlanta_df['hood_latitude']):
    try:
        hood_name = atlanta_df['Neighborhood'][index]
        latitude = atlanta_df['hood_latitude'][index]
        longitude = atlanta_df['hood_longitude'][index]
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
    except:
        print('No results')
    try:
        res = results['response']['venues']
    except:
        print('No venues')
        continue
    for dictionary in results['response']['venues']:
        try:
            ident= dictionary['id']
        except:
            ident= 'No ID'
        try:
            name = dictionary['name']
        except:
            name = 'No Name'
        try:
            ven_add = dictionary['location']['address']
        except:
            ven_add = 'No Address'
        try:
            ven_lat = dictionary['location']['lat']
        except:
            ven_lat = 'No Latitude'
        try:
            ven_lng = dictionary['location']['lng']
        except:
            ven_lng = 'No Longitude'
        try:
            ven_dist = dictionary['location']['distance']
        except:
            ven_dist = 'No Distance'
        try:
            for cat in dictionary['categories']:
                try:
                    cat_id= cat['id']
                except:
                    cat_id= 'No ID'
                try:
                    cat_name = cat['name']
                except:
                    cat_name = 'No Name'
        except:
                continue
        atlanta_venues = atlanta_venues.append({'Venue_Distance':ven_dist,'Hood':hood_name, 'Venue_ID':ident, 'Venue_Name':name,'Venue_Address':ven_add,'Venue_Latitude':ven_lat,'Venue_Longitude':ven_lng,'Category_ID':cat_id,'Category_Name':cat_name}, ignore_index=True)

No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues
No venues


In [43]:
atlanta_venues = atlanta_venues.set_index('Hood')
print(len(atlanta_venues))
atlanta_venues.head()

46


,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
Hood,,,,,,,,
Margaret Mitchell,552d475d498e4d32bce2696f,Atlanta Arcade & Gaming Rentals,705,925B Peachtree St NE,33.785318,-84.378605,4bf58dd8d48988d1e1931735,Arcade
Margaret Mitchell,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,2048,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
Margaret Mitchell,5b04dca96bd36b002c157123,Atlanta Arcade Rentals,1524,931 Monroe Dr NE #201,33.779940,-84.368230,5454152e498ef71e2b9132c6,Event Service
Ansley Park,552d475d498e4d32bce2696f,Atlanta Arcade & Gaming Rentals,1049,925B Peachtree St NE,33.785318,-84.378605,4bf58dd8d48988d1e1931735,Arcade
Ansley Park,5b04dca96bd36b002c157123,Atlanta Arcade Rentals,1790,931 Monroe Dr NE #201,33.779940,-84.368230,5454152e498ef71e2b9132c6,Event Service


In [46]:
print(len(atlanta_venues))
atlanta_venues.to_csv('{}_venue_data.csv'.format(search_query))

46


### Dropping the Nulls

In [47]:
atlanta_venues = pd.read_csv('{}_venue_data.csv'.format(search_query))
print(search_query)
atlanta_venues.head()

Arcade


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Margaret Mitchell,552d475d498e4d32bce2696f,Atlanta Arcade & Gaming Rentals,705,925B Peachtree St NE,33.785318,-84.378605,4bf58dd8d48988d1e1931735,Arcade
1,Margaret Mitchell,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,2048,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
2,Margaret Mitchell,5b04dca96bd36b002c157123,Atlanta Arcade Rentals,1524,931 Monroe Dr NE #201,33.779940,-84.368230,5454152e498ef71e2b9132c6,Event Service
3,Ansley Park,552d475d498e4d32bce2696f,Atlanta Arcade & Gaming Rentals,1049,925B Peachtree St NE,33.785318,-84.378605,4bf58dd8d48988d1e1931735,Arcade
4,Ansley Park,5b04dca96bd36b002c157123,Atlanta Arcade Rentals,1790,931 Monroe Dr NE #201,33.779940,-84.368230,5454152e498ef71e2b9132c6,Event Service


In [48]:
print(len(atlanta_venues))

46


In [49]:
atlanta_venues = atlanta_venues[atlanta_venues != 'Null']
atlanta_venues = atlanta_venues[atlanta_venues != '[]']
atlanta_venues = atlanta_venues.dropna()
print(len(atlanta_venues))
atlanta_venues = atlanta_venues.reset_index()
atlanta_venues = atlanta_venues.drop(['index'], axis = 1)
atlanta_venues.head()

46


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Margaret Mitchell,552d475d498e4d32bce2696f,Atlanta Arcade & Gaming Rentals,705,925B Peachtree St NE,33.785318,-84.378605,4bf58dd8d48988d1e1931735,Arcade
1,Margaret Mitchell,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,2048,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
2,Margaret Mitchell,5b04dca96bd36b002c157123,Atlanta Arcade Rentals,1524,931 Monroe Dr NE #201,33.779940,-84.368230,5454152e498ef71e2b9132c6,Event Service
3,Ansley Park,552d475d498e4d32bce2696f,Atlanta Arcade & Gaming Rentals,1049,925B Peachtree St NE,33.785318,-84.378605,4bf58dd8d48988d1e1931735,Arcade
4,Ansley Park,5b04dca96bd36b002c157123,Atlanta Arcade Rentals,1790,931 Monroe Dr NE #201,33.779940,-84.368230,5454152e498ef71e2b9132c6,Event Service


### Sorting by Venue and Distance

In [50]:
atlanta_sorted = atlanta_venues.sort_values(['Venue_ID','Venue_Distance'], ascending = True)
atlanta_sorted.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
32,Wildwood Forest,4c02e10d187ec92870e5b47b,Duffer's,1883,5210 Pacific Ave,38.980971,-74.825700,52e81612bcbc57f1066b79eb,Mini Golf
22,Old Fourth Ward,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,792,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
30,Inman Park,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,1450,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
12,Atkins Park,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,1498,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
14,Virginia Highland,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,1871,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office


In [51]:
atlanta_sorted['Category_Name'].unique()

array(['Mini Golf', 'Office', 'Arcade', 'Pub', 'Event Service'],
      dtype=object)

In [53]:
#didn't drop any venues
#it isn't clear if the other categories aren't arcades
atlanta_sorted['Category_Name'].unique()

array(['Mini Golf', 'Office', 'Arcade', 'Pub', 'Event Service'],
      dtype=object)

In [54]:
atlanta_sorted.to_csv('{}_sorted.csv'.format(search_query))

### Obtaining Neighborhood for Each Arcade

In [55]:
atlanta_sorted = pd.read_csv('{}_sorted.csv'.format(search_query))

In [56]:
atlanta_sorted = atlanta_sorted.drop('Unnamed: 0',axis =1 )

In [57]:
print(len(atlanta_sorted))
atlanta_sorted

46


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Wildwood Forest,4c02e10d187ec92870e5b47b,Duffer's,1883,5210 Pacific Ave,38.980971,-74.825700,52e81612bcbc57f1066b79eb,Mini Golf
1,Old Fourth Ward,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,792,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
2,Inman Park,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,1450,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
3,Atkins Park,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,1498,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
4,Virginia Highland,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,1871,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
5,Sweet Auburn,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,2025,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
6,Midtown,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,2026,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
7,Margaret Mitchell,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,2048,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
8,Summerhill,4d7412d3d6f06dcbbdb70f08,Foege's Arcade,428,No Address,33.740652,-84.384216,4bf58dd8d48988d1e1931735,Arcade
9,Capitol Gateway,4d7412d3d6f06dcbbdb70f08,Foege's Arcade,575,No Address,33.740652,-84.384216,4bf58dd8d48988d1e1931735,Arcade


In [58]:
count = -1
list_to_drop = list()
for ID in atlanta_sorted['Venue_ID']:
    count = count +1
    try:
        if atlanta_sorted['Venue_ID'][count] == atlanta_sorted['Venue_ID'][count-1] and atlanta_sorted['Venue_Distance'][count]>=atlanta_sorted['Venue_Distance'][count-1]:
            list_to_drop.append(count)
    except:
        continue
list_to_drop

[2,
 3,
 4,
 5,
 6,
 7,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 41,
 42,
 43,
 44,
 45]

In [59]:
atlanta_final = atlanta_sorted.drop(index = list_to_drop)

In [60]:
print(len(atlanta_final))
atlanta_final

7


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Wildwood Forest,4c02e10d187ec92870e5b47b,Duffer's,1883,5210 Pacific Ave,38.980971,-74.825700,52e81612bcbc57f1066b79eb,Mini Golf
1,Old Fourth Ward,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,792,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
8,Summerhill,4d7412d3d6f06dcbbdb70f08,Foege's Arcade,428,No Address,33.740652,-84.384216,4bf58dd8d48988d1e1931735,Arcade
19,Lake Clair,4f5c6a22e4b0a3bd23833b66,Marlin Arcade,1142,No Address,38.495137,-90.007739,4bf58dd8d48988d11b941735,Pub
20,Downtown,503e646b16488fc60417dc31,PAC-MAN PLAY™ Arcade at Underground Atlanta,322,50 Upper Alabama Street,33.752497,-84.390085,4bf58dd8d48988d1e1931735,Arcade
30,Midtown,552d475d498e4d32bce2696f,Atlanta Arcade & Gaming Rentals,649,925B Peachtree St NE,33.785318,-84.378605,4bf58dd8d48988d1e1931735,Arcade
40,Virginia Highland,5b04dca96bd36b002c157123,Atlanta Arcade Rentals,1378,931 Monroe Dr NE #201,33.779940,-84.368230,5454152e498ef71e2b9132c6,Event Service


In [61]:
atlanta_final.to_csv('{}_unique.csv'.format(search_query))

### Pulling the Ratings

In [62]:
search_query

'Arcade'

In [63]:
atlanta_final = pd.read_csv('{}_unique.csv'.format(search_query))
atlanta_final.head()

,Unnamed: 0,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,0,Wildwood Forest,4c02e10d187ec92870e5b47b,Duffer's,1883,5210 Pacific Ave,38.980971,-74.825700,52e81612bcbc57f1066b79eb,Mini Golf
1,1,Old Fourth Ward,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,792,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
2,8,Summerhill,4d7412d3d6f06dcbbdb70f08,Foege's Arcade,428,No Address,33.740652,-84.384216,4bf58dd8d48988d1e1931735,Arcade
3,19,Lake Clair,4f5c6a22e4b0a3bd23833b66,Marlin Arcade,1142,No Address,38.495137,-90.007739,4bf58dd8d48988d11b941735,Pub
4,20,Downtown,503e646b16488fc60417dc31,PAC-MAN PLAY™ Arcade at Underground Atlanta,322,50 Upper Alabama Street,33.752497,-84.390085,4bf58dd8d48988d1e1931735,Arcade


In [64]:
atlanta_final = atlanta_final.drop('Unnamed: 0', axis = 1)
atlanta_final.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Wildwood Forest,4c02e10d187ec92870e5b47b,Duffer's,1883,5210 Pacific Ave,38.980971,-74.825700,52e81612bcbc57f1066b79eb,Mini Golf
1,Old Fourth Ward,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,792,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office
2,Summerhill,4d7412d3d6f06dcbbdb70f08,Foege's Arcade,428,No Address,33.740652,-84.384216,4bf58dd8d48988d1e1931735,Arcade
3,Lake Clair,4f5c6a22e4b0a3bd23833b66,Marlin Arcade,1142,No Address,38.495137,-90.007739,4bf58dd8d48988d11b941735,Pub
4,Downtown,503e646b16488fc60417dc31,PAC-MAN PLAY™ Arcade at Underground Atlanta,322,50 Upper Alabama Street,33.752497,-84.390085,4bf58dd8d48988d1e1931735,Arcade


In [65]:
count = -1
rating_list = list()
for ident in atlanta_final['Venue_ID']:
    count = count +1
    venue_id = atlanta_final['Venue_ID'][count]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    try:
        rating = result['response']['venue']['rating']
    except:
        rating = 'Not Rated'
    rating_list.append(rating)
rating_list

[8.5,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated']

In [66]:
atlanta_final = atlanta_final.assign(Venue_Rating = rating_list)

In [67]:
atlanta_final.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,Wildwood Forest,4c02e10d187ec92870e5b47b,Duffer's,1883,5210 Pacific Ave,38.980971,-74.825700,52e81612bcbc57f1066b79eb,Mini Golf,8.5
1,Old Fourth Ward,4d6678a3153ba0930ff4e9f8,Arcade 160 Studios,792,621 North Ave NE Ste C,33.770101,-84.367149,4bf58dd8d48988d124941735,Office,Not Rated
2,Summerhill,4d7412d3d6f06dcbbdb70f08,Foege's Arcade,428,No Address,33.740652,-84.384216,4bf58dd8d48988d1e1931735,Arcade,Not Rated
3,Lake Clair,4f5c6a22e4b0a3bd23833b66,Marlin Arcade,1142,No Address,38.495137,-90.007739,4bf58dd8d48988d11b941735,Pub,Not Rated
4,Downtown,503e646b16488fc60417dc31,PAC-MAN PLAY™ Arcade at Underground Atlanta,322,50 Upper Alabama Street,33.752497,-84.390085,4bf58dd8d48988d1e1931735,Arcade,Not Rated


In [68]:
atlanta_final.to_csv('{}_final_clean.csv'.format(search_query))